<a href="https://colab.research.google.com/github/Intelligent-Systems-Phystech/2022-Project-93-2/blob/master/code/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 55 kB/s 


In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import make_pipeline
import time
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, auc, average_precision_score, \
                            make_scorer, precision_recall_curve, roc_curve
from sklearn.metrics import precision_score, recall_score   
from collections import Counter
from google.colab import drive
from datetime import datetime
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix
from lightgbm import LGBMClassifier

In [4]:
from google.colab import drive
drive.mount('/content/drive')
dir_path = '/content/drive/My Drive/Skoltech Floods/'
import os
import sys
sys.path.append(dir_path)

Mounted at /content/drive


In [5]:
ex_dates = pd.read_table(dir_path + 'exday_99per_DJF_CASM_1980_2019.txt', delim_whitespace=True, names=('day_of_seq', 'year', 'month', 'day'))
nex_dates = pd.read_table(dir_path + 'nexday_99per_DJF_CASM_1980_2019.txt', delim_whitespace=True, names=('day_of_seq', 'year', 'month', 'day'))
ex_dates.insert(0, 'ex', pd.Series([1]*len(ex_dates)))
nex_dates.insert(0, 'ex', pd.Series([0]*len(nex_dates)))

features = ['h500', 'ivt', 'qv2m', 'slp', 'tpw', 'uqv', 'vqv', 'w500']
mean_vals = dict()

for feature in features:
    ex_X = pd.read_table(dir_path + f'ERA5/CASM/ERA5_DJF_CASM_99per_ex_n{feature}_1980_2019.txt', delim_whitespace=True, header = None)
    nex_X = pd.read_table(dir_path + f'ERA5/CASM/ERA5_DJF_CASM_99per_nex_n{feature}_1980_2019.txt', delim_whitespace=True, header = None)
    ex = pd.concat([ex_dates, ex_X], axis = 1)
    nex = pd.concat([nex_dates, nex_X], axis = 1)
    X_mean = pd.concat([ex, nex]).sort_values('day_of_seq').reset_index(drop=True)
    mean = X_mean.iloc[:,5:].values.mean(axis=1)
    mean_vals[feature] = mean

In [6]:
ex_dates = pd.read_table(dir_path + 'exday_99per_DJF_CASM_1980_2019.txt', delim_whitespace=True, names=('day_of_seq', 'year', 'month', 'day'))
nex_dates = pd.read_table(dir_path + 'nexday_99per_DJF_CASM_1980_2019.txt', delim_whitespace=True, names=('day_of_seq', 'year', 'month', 'day'))
ex_dates.insert(0, 'ex', pd.Series([1]*len(ex_dates)))
nex_dates.insert(0, 'ex', pd.Series([0]*len(nex_dates)))

features = ['h500', 'ivt', 'qv2m', 'slp', 'tpw', 'uqv', 'vqv', 'w500']
std_vals = dict()

for feature in features:
    ex_X = pd.read_table(dir_path + f'ERA5/CASM/ERA5_DJF_CASM_99per_ex_n{feature}_1980_2019.txt', delim_whitespace=True, header = None)
    nex_X = pd.read_table(dir_path + f'ERA5/CASM/ERA5_DJF_CASM_99per_nex_n{feature}_1980_2019.txt', delim_whitespace=True, header = None)
    ex = pd.concat([ex_dates, ex_X], axis = 1)
    nex = pd.concat([nex_dates, nex_X], axis = 1)
    X_std = pd.concat([ex, nex]).sort_values('day_of_seq').reset_index(drop=True)
    std = X_std.iloc[:,5:].values.std(axis=1)
    std_vals[feature] = std

In [7]:
X_mean['date'] = pd.to_datetime(X_mean[['year', 'month', 'day']])
df_mean = pd.DataFrame.from_dict(mean_vals)
df_mean['date'] = X_mean.date
df_mean.set_index('date', inplace = True, drop = True)
df_mean['ex'] = X_mean['ex'].values
df_mean.columns = ['h500_m', 'ivt_m', 'qv2m_m', 'slp_m', 'tpw_m', 'uqv_m', 'vqv_m', 'w500_m', 'ex']
df_mean.drop(columns = 'ex', inplace = True)

X_std['date'] = pd.to_datetime(X_std[['year', 'month', 'day']])
df_std = pd.DataFrame.from_dict(std_vals)
df_std['date'] = X_std.date
df_std.set_index('date', inplace = True, drop = True)
df_std['ex'] = X_std['ex'].values
df_std.columns = ['h500_std', 'ivt_std', 'qv2m_std', 'slp_std', 'tpw_std', 'uqv_std', 'vqv_std', 'w500_std', 'ex']

df = pd.concat([df_mean, df_std], axis = 1)

In [8]:
df = df.reset_index()
df["day"] = df["date"].dt.dayofyear

In [10]:
df['day_of_year'] = (df["day"] + 31) % (335 + 31)

In [11]:
df_no_std = df.drop(["day", "date", "h500_std",	"ivt_std",	"qv2m_std", "slp_std",	"tpw_std",	"uqv_std",	"vqv_std",	"w500_std"], axis=1)

In [12]:
to_shift = df_no_std.columns[:-2]

In [13]:
for feature in to_shift:
  for i in range(1, 4):
    df_no_std[feature + str(-i)] = df_no_std[feature].shift(i)

In [15]:
df_no_std = df_no_std.drop([df_no_std.index[0] , df_no_std.index[1], df_no_std.index[2]])

In [16]:
df_no_std

,h500_m,ivt_m,qv2m_m,slp_m,tpw_m,uqv_m,vqv_m,w500_m,ex,day_of_year,...,tpw_m-3,uqv_m-1,uqv_m-2,uqv_m-3,vqv_m-1,vqv_m-2,vqv_m-3,w500_m-1,w500_m-2,w500_m-3
3,-0.490461,-0.315219,-0.333487,-0.375833,-0.181184,-0.119781,-0.086184,-0.056316,0,35,...,0.303355,-0.087368,-0.104342,-0.163311,0.013048,0.238004,0.322961,-0.151075,-0.172237,-0.190548
4,-0.530000,-0.075943,-0.423838,-0.501952,-0.127149,0.031535,0.107346,-0.132105,0,36,...,0.250636,-0.119781,-0.087368,-0.104342,-0.086184,0.013048,0.238004,-0.056316,-0.151075,-0.172237
5,-0.447346,-0.014737,-0.298947,-0.293465,-0.122763,0.056864,0.028706,-0.131798,0,37,...,0.018004,0.031535,-0.119781,-0.087368,0.107346,-0.086184,0.013048,-0.132105,-0.056316,-0.151075
6,-0.668465,-0.103816,-0.161798,-0.213092,-0.072785,0.136096,-0.137785,-0.129583,0,38,...,-0.181184,0.056864,0.031535,-0.119781,0.028706,0.107346,-0.086184,-0.131798,-0.132105,-0.056316
7,-0.872390,-0.209868,-0.179035,-0.162237,-0.150373,0.186776,-0.153596,0.142478,0,39,...,-0.127149,0.136096,0.056864,0.031535,-0.137785,0.028706,0.107346,-0.129583,-0.131798,-0.132105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3605,-0.175351,-0.014452,0.167785,-0.391974,0.374825,0.219232,0.174035,-0.075000,0,26,...,0.089825,0.094978,0.083860,0.231491,-0.006316,0.024803,0.313553,0.015614,-0.002149,0.040680
3606,-0.092259,0.247281,0.184232,-0.134211,0.361140,0.244408,0.149496,0.022939,0,27,...,0.146754,0.219232,0.094978,0.083860,0.174035,-0.006316,0.024803,-0.075000,0.015614,-0.002149
3607,0.118947,0.122281,0.104649,0.306382,0.121842,0.032127,-0.331382,0.303882,0,28,...,0.143224,0.244408,0.219232,0.094978,0.149496,0.174035,-0.006316,0.022939,-0.075000,0.015614
3608,0.397171,0.292785,0.161206,0.354846,0.254408,0.188180,-0.247193,0.229408,0,29,...,0.374825,0.032127,0.244408,0.219232,-0.331382,0.149496,0.174035,0.303882,0.022939,-0.075000


In [17]:
def scores(y_actual, y_predicted):
    accuracy = accuracy_score(y_actual, y_predicted)
    precision = precision_score(y_actual, y_predicted)
    recall = recall_score(y_actual, y_predicted)
    return accuracy, precision, recall


def conf(y_test, y_pred):
  return confusion_matrix(y_test, y_pred)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df_no_std.drop(["ex"], axis=1), df_no_std["ex"], stratify = df_no_std["ex"], test_size=0.25)

Let's prepare our data with methods SMOTE and UnderSampling, also left original data

In [35]:
smote = SMOTE(k_neighbors = 7)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [70]:
undersample = RandomUnderSampler(sampling_strategy='majority')
X_train_under, y_train_under = undersample.fit_resample(X_train, y_train)

Now let's try our models on these train samples

In [125]:
results = pd.DataFrame()

In [55]:
feature_name_list = ['accuracy', 'precision', 'recall']

Catboost

In [126]:
model = CatBoostClassifier(verbose=0)
model.fit(X_train, y_train)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'CatBoost'
dct_['sampling'] = None                                          
results = results.append(dct_, ignore_index=True)

In [127]:
model.fit(X_train_smote, y_train_smote)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'CatBoost'
dct_['sampling'] = 'Smote'                                          
results = results.append(dct_, ignore_index=True)

In [128]:
model.fit(X_train_under, y_train_under)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'CatBoost'
dct_['sampling'] = 'Under'                                          
results = results.append(dct_, ignore_index=True)

SVC-linear

In [129]:
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'SVC-linear'
dct_['sampling'] = None                                          
results = results.append(dct_, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [130]:
model.fit(X_train_smote, y_train_smote)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'SVC-linear'
dct_['sampling'] = 'Smote'                                          
results = results.append(dct_, ignore_index=True)

In [131]:
model.fit(X_train_under, y_train_under)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'SVC-linear'
dct_['sampling'] = 'Under'                                          
results = results.append(dct_, ignore_index=True)

SVC-kbf

In [132]:
model = SVC(kernel='rbf', probability=True)
model.fit(X_train, y_train)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'SVC-rbf'
dct_['sampling'] = None                                          
results = results.append(dct_, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [133]:
model.fit(X_train_smote, y_train_smote)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'SVC-rbf'
dct_['sampling'] = 'Smote'                                          
results = results.append(dct_, ignore_index=True)

In [134]:
model.fit(X_train_under, y_train_under)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'SVC-rbf'
dct_['sampling'] = 'Under'                                          
results = results.append(dct_, ignore_index=True)

Logistic Regression

In [135]:
model = LogisticRegression(penalty='l2')
model.fit(X_train, y_train)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'LogReg'
dct_['sampling'] = None                                          
results = results.append(dct_, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [136]:
model.fit(X_train_smote, y_train_smote)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'LogReg'
dct_['sampling'] = 'Smote'                                          
results = results.append(dct_, ignore_index=True)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [137]:
model.fit(X_train_under, y_train_under)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'LogReg'
dct_['sampling'] = 'Under'                                          
results = results.append(dct_, ignore_index=True)

Random Forest

In [138]:
model = RandomForestClassifier(n_estimators=200)
model.fit(X_train, y_train)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'RandomForest'
dct_['sampling'] = None                                          
results = results.append(dct_, ignore_index=True)

In [139]:
model.fit(X_train_smote, y_train_smote)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'RandomForest'
dct_['sampling'] = 'Smote'                                          
results = results.append(dct_, ignore_index=True)

In [140]:
model.fit(X_train_under, y_train_under)
dct_ = {name:value for name, value in zip(feature_name_list, scores(y_test, model.predict(X_test)))}
dct_['conf_mat'] = conf(y_test, model.predict(X_test))
dct_['name'] = 'RandomForest'
dct_['sampling'] = 'Under'                                          
results = results.append(dct_, ignore_index=True)

In [141]:
results

,accuracy,precision,recall,conf_mat,name,sampling
0,0.980044,1.000000,0.052632,"[[883, 0], [18, 1]]",CatBoost,None
1,0.949002,0.170732,0.368421,"[[849, 34], [12, 7]]",CatBoost,Smote
2,0.733925,0.066667,0.894737,"[[645, 238], [2, 17]]",CatBoost,Under
3,0.978936,0.000000,0.000000,"[[883, 0], [19, 0]]",SVC-linear,None
4,0.829268,0.090909,0.789474,"[[733, 150], [4, 15]]",SVC-linear,Smote
5,0.770510,0.076577,0.894737,"[[678, 205], [2, 17]]",SVC-linear,Under
6,0.978936,0.000000,0.000000,"[[883, 0], [19, 0]]",SVC-rbf,None
7,0.746120,0.066116,0.842105,"[[657, 226], [3, 16]]",SVC-rbf,Smote
8,0.427938,0.015595,0.421053,"[[378, 505], [11, 8]]",SVC-rbf,Under
9,0.977827,0.333333,0.052632,"[[881, 2], [18, 1]]",LogReg,None


Long-run part.



In [ ]:
search_cat = RandomizedSearchCV(
      estimator=LGBMClassifier(verbose=0),
      param_distributions={
          "reg_lambda": np.logspace(-4, 0, 30),
          "n_estimators" : np.arange(450, 1000, 20),
          "learning_rate" : np.linspace(0.001, 0.02, 40),
          "max_depth": np.arange(4, 20)
      },
      scoring='recall',
      n_iter=25,
      n_jobs=-1
)
search_cat.fit(X_train_smote, y_train_smote)